In [1]:
import sys, psycopg2
sys.path.append('..')
from utils.redis_utils import * 
redis_client = RedisUtils(RedisType.CLUSTER).redis_client
conn = psycopg2.connect(
    dbname="duongnka",
    user="postgres",
    password="Aa123456",
    host="localhost"
)
cursor = conn.cursor()


In [ ]:
MAX_CACHE_SIZE = 15

# define essential functions

def fetch_data_from_postgres(product_id):
    query = f"SELECT * FROM products WHERE id = {product_id}"
    cursor.execute(query)
    return cursor.fetchone()

def get_product_info(product_id):
    cached_data = redis_client.get(f"product:{product_id}")
    if cached_data:
        print("Data found in Redis cache!")
        return cached_data.decode('utf-8')
    
    print("Fetching data from PostgreSQL...")
    data = fetch_data_from_postgres(product_id)

    redis_client.execute_command(f"product:{product_id}", 3600, str(data))

    if redis_client.dbsize() > MAX_CACHE_SIZE:
        oldest_key = redis_client.execute_command("LINDEX", 'product_keys', -1)
        redis_client.delete(oldest_key)
        redis_client.execute_command("LPOP", "product_keys")
    
    redis_client.rpush("product_keys", f"product:{product_id}")

    return data

def invalidate_product_cache(product_id):
    redis_client.delete(f"product:{product_id}")
    redis_client.lrem("product_keys", 0, f"product:{product_id}")
    print(f"cache for product {product_id} invalidated.")

def add_new_product(name, price, code):
    insert_query = f"INSERT INTO products (name, price, code) VALUES ({name}, {price}, {code})"
    cursor.execute(insert_query)
    new_product_id = cursor.fetchone()[0]
    conn.commit()

    invalidate_product_cache(new_product_id)
    print(f"New product added with ID {new_product_id}.")

    return new_product_id

def update_product_price(product_id, new_price):
    update_query = f"UPDATE products SET price = {new_price} WHERE id = {product_id}"
    cursor.execute(update_query)
    conn.commit()

    invalidate_product_cache(product_id)
    print(f"Product {product_id} price updated.")


In [2]:
# mock up data
import random, string, time

def generate_random_name():
    prefixes = ["Cool", "Amazing", "Fantastic", "Awesome"]
    suffixes = ["Gadget", "Widget", "Device", "Tool"]
    prefix = random.choice(prefixes)
    suffix = random.choice(suffixes)
    return f"{prefix} {suffix}"

def generate_random_price():
    return round(random.uniform(10.0, 1000.0), 2)

def generate_random_code(length=10):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for _ in range(length)) 

def generate_mock_product_data():
    name = generate_random_name()
    price = generate_random_price()
    code = generate_random_code()
    return name, price, code

def insert_mock_products_batch(num_products=100000, batch_size=10000):
    num_batches = (num_products + batch_size - 1) // batch_size
    remaining_products = num_products

    for batch_num in range(num_batches):
        current_batch_size = min(batch_size, remaining_products)
        mock_data = [generate_mock_product_data() for _ in range(current_batch_size)]
        insert_query = "INSERT INTO products (name, price, code) VALUES (%s, %s, %s)"
        print(mock_data[0], '...')
        start_time = time.time()
        cursor.executemany(insert_query, mock_data)
        conn.commit()
        end_time = time.time()
        print(f"Generated and inserted {current_batch_size} mock products in {end_time - start_time:.2f} seconds.")
        print(f"Inserted batch {batch_num + 1} / {num_batches}")

        remaining_products -= current_batch_size
        if remaining_products <= 0:
            break

insert_mock_products_batch(num_products=123241)




('Awesome Device', 71.05, 'jorsjzjmol') ...
Generated and inserted 10000 mock products in 4.05 seconds.
Inserted batch 1 / 13
('Awesome Device', 91.9, 'nrtvzvyfhp') ...
Generated and inserted 10000 mock products in 4.11 seconds.
Inserted batch 2 / 13
('Fantastic Tool', 555.88, 'gsusyhrzgz') ...
Generated and inserted 10000 mock products in 4.07 seconds.
Inserted batch 3 / 13
('Fantastic Tool', 620.11, 'fndrduzwqe') ...
Generated and inserted 10000 mock products in 4.02 seconds.
Inserted batch 4 / 13
('Amazing Gadget', 747.35, 'bmkescfjms') ...
Generated and inserted 10000 mock products in 3.98 seconds.
Inserted batch 5 / 13
('Cool Gadget', 823.84, 'wukmuczmqe') ...
Generated and inserted 10000 mock products in 4.02 seconds.
Inserted batch 6 / 13
('Cool Widget', 571.75, 'noqdqwsmlw') ...
Generated and inserted 10000 mock products in 4.04 seconds.
Inserted batch 7 / 13
('Awesome Gadget', 979.25, 'hjvzjthzcg') ...
Generated and inserted 10000 mock products in 4.03 seconds.
Inserted batch 